In [ ]:
import os
import json

In [ ]:
def json_to_yolo(json_file, output_folder, label_mapping, image_folder):
    # JSON 파일을 읽어옵니다.
    with open(json_file) as f:
        data = json.load(f)

    # 이미지 정보와 파일 이름을 가져옵니다.
    image_info = data["images"]
    image_name = image_info["file_name"].split('.')[0]
    yolo_labels = []

    # JSON 데이터의 각 주석에 대해 반복합니다.
    for annotation_info in data["annotations"]:
        # 카테고리 ID를 가져옵니다.
        category_id = annotation_info["category_id"]

        # 레이블 매핑에 있는 카테고리가 있으면 해당 정보를 이용해 YOLO 형식으로 변환합니다.
        if category_id in label_mapping:
            bbox = annotation_info["bbox"]  # bounding box 좌표를 가져옵니다.
            xmin, ymin, width, height = bbox  # bounding box의 xmin, ymin, 폭, 높이를 가져옵니다.

            # 클래스 ID를 레이블 매핑에서 가져옵니다.
            class_id = label_mapping[category_id]

            try:
                image_info["width"] = 1280
                image_info["height"] = 720
                # 카테고리와 관련된 좌표를 YOLO format으로 변환합니다.
                x_center = (xmin + width / 2) / image_info["width"]
                y_center = (ymin + height / 2) / image_info["height"]
                box_width = width / image_info["width"]
                box_height = height / image_info["height"]
            except ZeroDivisionError:
                print(f"오류: {json_file} 이미지의 width 또는 height 값이 0입니다.")
                return

            # YOLO 형식 레이블을 추가합니다.
            yolo_labels.append(f"{class_id} {x_center} {y_center} {box_width} {box_height}")

    output_file_path = os.path.join(output_folder, f"{image_name}.txt")

    # YOLO 형식 레이블을 txt 파일로 저장합니다.
    with open(output_file_path, "w") as f:
        f.write("\n".join(yolo_labels))

    # 결과 값 용량이 0KB인 경우 파일을 삭제하고 삭제한 파일 이름을 반환합니다.
    if os.path.getsize(output_file_path) == 0:
        os.remove(output_file_path)
        # 이미지 파일 삭제 부분 추가
        image_file_path = None
        for image_ext in [".png"]:
            test_path = os.path.join(image_folder, f"{image_name}{image_ext}")
            if os.path.exists(test_path):
                image_file_path = test_path
                break
        if image_file_path is not None:
            os.remove(image_file_path)
        return image_name
    return None

# 카테고리 매핑.
label_mapping = {1: 0, 3: 1, 6: 2, 7: 3}

# JSON 파일과 출력 디렉터리 경로를 지정합니다.
input_folder = 'MMPP_data/Validation/Annotations/TOA/5.Mainroad_F01/'
output_folder = "MMPP_data/5.Mainroad_F01_del"

# 이미지 디렉터리 경로를 지정합니다.
image_folder = "MMPP_data/Validation/Images/TOA/5.Mainroad_F01/"

# 출력 디렉터리가 없을 경우 생성합니다.
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

json_files = [f for f in os.listdir(input_folder) if f.endswith('.json')]

# 삭제된 파일 목록을 저장할 리스트를 생성합니다.
deleted_files = []

for json_file in json_files:
    json_file_path = os.path.join(input_folder, json_file)
    # image_folder 매개변수 추가
    deleted_file = json_to_yolo(json_file_path, output_folder, label_mapping, image_folder)

    # 반환된 파일 이름이 None이 아니면 삭제된 파일이므로 목록에 추가합니다.
    if deleted_file:
        deleted_files.append(deleted_file)

print("삭제된 파일 목록:")
for deleted_file in deleted_files:
    print(deleted_file)